In [1]:
from flask import Flask, jsonify, redirect, request
from flask_cors import CORS
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from konlpy.tag import Okt
import json

#커스텀 모델 호출을 위한 함수 선언
from keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림 - 0.5 이하 = 0 / 0.5 초과 = 1
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정
    # True Positive = 실제 값과 예측 값이 모두 1(Positive)인 경우
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)
    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon() = 'divide by zero error' 예방차원에서 작은 수를 더함
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    # return a single tensor value
    return recall
def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림 - 0.5 이하 = 0 / 0.5 초과 = 1
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정
    # True Positive = 실제 값과 예측 값이 모두 1(Positive)인 경우
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)
    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon() = 'divide by zero error' 예방차원에서 작은 수를 더함
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())
    # return a single tensor value
    return precision
def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon() = divide by zero error' 예방차원에서 작은 수를 더함
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    # return a single tensor value
    return _f1score

# Server
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

# 문장 및 사용될 단어의 최대 길이/갯수 설정
maxlen = 500
max_words = 2000001
# 생성된 형태소 배열 토큰화 및 정수 인코딩
tokenizer = Tokenizer(num_words=max_words)
with open('model\wordIndex_abusive.json') as json_file:
	word_index = json.load(json_file)
	tokenizer.word_index = word_index
tokenizer.fit_on_texts(word_index) # 빈도수 기준으로 단어 집합 생성
    
# 여기다 모델 3개 로딩예정
model_c1 = load_model('model\cnn-lstm\model_abusive_cl.h5', custom_objects = {"precision": precision, "recall" : recall, "f1score" : f1score})
model_c2 = load_model('model\cnn-lstm\model_slang_cl.h5', custom_objects = {"precision": precision, "recall" : recall, "f1score" : f1score})
model_c3 = load_model('model\cnn-lstm\model_sexual_cl.h5', custom_objects = {"precision": precision, "recall" : recall, "f1score" : f1score})


# Ai Function 
@app.route("/use_model", methods=['POST'])
def use_model():
	# Chrome Extension으로부터 전송받은 데이터
	examples_temp = request.get_json()
	examples = examples_temp["text"]
	check = examples_temp["check"]  # check[0] ~ [2] = 욕설, 비속어, 성적표현
	c1 = float(check[0] == True)
	c2 = float(check[1] == True)
	c3 = float(check[2] == True)
	print(check)
	okt = Okt()
	ex_morpheme = []
	# 문장 형태소 분석
	for text in examples:
		mor_data = ""
		for word_tag in okt.pos(text, norm=True, stem=True):
			if word_tag[1] in ['Noun', 'Verb', 'VerbPrefix', 'Adjective', 'Determiner', 'Adverb', 'Exclamation', 'KoreanParticle']:
				mor_data += word_tag[0]
				mor_data += " "
		ex_morpheme.append(mor_data)
	sequences = tokenizer.texts_to_sequences(ex_morpheme) # 단어 빈도수가 높은 순으로 번호 부여
	# 문장 별 길이를 동일하게 맞추기 위한 패딩
	x_test = pad_sequences(sequences, maxlen=maxlen)
	
    #데이터로부터 욕설 가능성 계산 후 반환
	prob_c1 = model_c1.predict(x_test)
	prob_c2 = model_c2.predict(x_test)
	prob_c3 = model_c3.predict(x_test)
    
	labels_c1 = [] #욕설 데이터 값 저장용
	for i in range(0, len(x_test)):
		labels_c1.extend([round(prob_c1[i][0] * 100, 1)])
	labels_c2 = [] #비속어 데이터 값 저장용
	for i in range(0, len(x_test)):
		labels_c2.extend([round(prob_c2[i][0] * 100, 1)])
	labels_c3 = [] #성적 표현 데이터 값 저장용
	for i in range(0, len(x_test)):
		labels_c3.extend([round(prob_c3[i][0] * 100, 1)])
    
    #확률 계산
	prob_check = []
	print(len(x_test))
	for i in range(0, len(x_test)):
		prob_check.append(max(labels_c1[i]*c1, labels_c2[i]*c2, labels_c3[i]*c3))
	for i in range(0, len(x_test)):
		print(examples[i], ":", prob_check[i], "%의 확률로 필터링 대상입니다.")            
	prob_json = { "prob": []}
	for i in range(0, len(x_test)):
		prob_json["prob"].append(float(prob_check[i]))
	return prob_json

# Server address
if __name__ == "__main__":
	app.run(host="127.0.0.1", port="5000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
